데이터가 나누어져 있을 때 데이터를 합치거나 연결해야 하는 경우가 있다. 이를 위해 판다스의 concat(), merge(), join() 함수를 사용할 수 있다.

<h3> 데이터프레임 연결 </h3>

데이터프레임들을 기존 형태를 유지하면서 이어 붙이기 위해 **concat() 함수**를 사용할 수 있다. 

* **데이터프레임 연결: pandas.concat(데이터프레임의 리스트)**

concat() 함수에 데이터프레임을 원소로 갖는 리스트를 전달하면 여러 개의 데이터프레임을 연결한다. <br>
**axis 매개변수**의 기본값은 0이며 위 아래 행 방향으로 연결된다. 이때 각 데이터프레임의 행 인덱스는 본래 형태를 유지한다. <br>
**join 매개변수**를 **'outer'**로 지정하면 열 이름의 합집합으로 데이터프레임을 연결한다. 예를 들어, df1의 열 이름이 A, B, C고 df2의 열 이름이 B, C, D, E이면 A, B, C, D, E의 열로 구성된다. **'inner'**로 지정할 경우 교집합으로 연결하여 위 경우 B, C로 구성된다. 기본값은 'outer'이다. <br>
먼저 'outer' 방식으로 연결해보자.

In [ ]:
import pandas as pd

df1 = pd.DataFrame({'a': ['a0', 'a1', 'a2', 'a3'],
                    'b': ['b0', 'b1', 'b2', 'b3'],
                    'c': ['c0', 'c1', 'c2', 'c3']}, 
                    index = [0, 1, 2, 3])

df2 = pd.DataFrame({'a': ['a2', 'a3', 'a4', 'a5'],
                    'b': ['b2', 'b3', 'b4', 'b5'],
                    'c': ['c2', 'c3', 'c4', 'c5'],
                    'd': ['d2', 'd3', 'd4', 'd5']},
                    index = [2, 3, 4, 5])

print(df1, '\n')
print(df2, '\n')

result1 = pd.concat([df1, df2])
print(result1)

    a   b   c
0  a0  b0  c0
1  a1  b1  c1
2  a2  b2  c2
3  a3  b3  c3 

    a   b   c   d
2  a2  b2  c2  d2
3  a3  b3  c3  d3
4  a4  b4  c4  d4
5  a5  b5  c5  d5 

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
2  a2  b2  c2   d2
3  a3  b3  c3   d3
4  a4  b4  c4   d4
5  a5  b5  c5   d5


겹치는 행 인덱스 두 개를 모두 접근하려면 iloc() 메서드를 사용하면 된다.

In [ ]:
print(result1.index, '\n')
print(result1.loc[2], '\n')
print(result1.iloc[2])

Int64Index([0, 1, 2, 3, 2, 3, 4, 5], dtype='int64') 

    a   b   c    d
2  a2  b2  c2  NaN
2  a2  b2  c2   d2 

a     a2
b     b2
c     c2
d    NaN
Name: 2, dtype: object


위와 같이 행 인덱스가 겹치는 것을 방지하려면 기존 행 인덱스를 없애면 된다. <br>
**ignore_index 매개변수**를 True로 지정하면 기존 행 인덱스를 무시한다.

In [ ]:
result2 = pd.concat([df1, df2], ignore_index = True)
print(result2)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
4  a2  b2  c2   d2
5  a3  b3  c3   d3
6  a4  b4  c4   d4
7  a5  b5  c5   d5


**axis 매개변수를 1로 지정**하면 좌우 열 방향으로 연결한다. 따라서 기존 열 이름이 그대로 유지된다. 행 인덱스는 join 매개변수가 'outer'면 합집합으로 구성된다. 물론 기본값은 'outer'이다.

In [ ]:
result3 = pd.concat([df1, df2], axis = 1)
print(result3)

     a    b    c    a    b    c    d
0   a0   b0   c0  NaN  NaN  NaN  NaN
1   a1   b1   c1  NaN  NaN  NaN  NaN
2   a2   b2   c2   a2   b2   c2   d2
3   a3   b3   c3   a3   b3   c3   d3
4  NaN  NaN  NaN   a4   b4   c4   d4
5  NaN  NaN  NaN   a5   b5   c5   d5


이번에는 join 매개변수를 'inner'로 지정해보자. 행 인덱스의 교집합을 기준으로 연결하므로 2, 3 행만 남는다.

In [ ]:
result3_in = pd.concat([df1, df2], axis = 1, join = 'inner')
print(result3_in)

    a   b   c   a   b   c   d
2  a2  b2  c2  a2  b2  c2  d2
3  a3  b3  c3  a3  b3  c3  d3


**데이터프레임과 시리즈를 좌우 열 방향으로 연결**할 수도 있다. 이때 데이터프레임의 행 인덱스와 시리즈의 인덱스가 같아야한다. <br>
데이터프레임에 열을 추가하는 것과 동일한 결과를 가지며 열의 이름은 시리즈의 이름으로 변환된다.

* 행 인덱스가 같지 않으면 NaN 값으로 입력된다.

In [ ]:
sr1 = pd.Series(['e0', 'e1', 'e2', 'e3'], name = 'e')
sr2 = pd.Series(['f0', 'f1', 'f2'], name = 'f', index = [3, 4, 5])
sr3 = pd.Series(['g0', 'g1', 'g2', 'g3'], name = 'g')
sr4 = pd.Series(['h0', 'h1', 'h2', 'h3'], name = 'h', index = [1, 2, 3, 4])

result4 = pd.concat([df1, sr1], axis = 1)
print(result4, '\n')

result5 = pd.concat([df2, sr2], axis = 1, sort = False)
print(result5, '\n')

result5_sort = pd.concat([df2, sr2], axis = 1, sort = True)
print(result5_sort, '\n')

result6 = pd.concat([df1, sr4], axis = 1)
print(result6)

    a   b   c   e
0  a0  b0  c0  e0
1  a1  b1  c1  e1
2  a2  b2  c2  e2
3  a3  b3  c3  e3 

    a   b   c   d    f
2  a2  b2  c2  d2  NaN
3  a3  b3  c3  d3   f0
4  a4  b4  c4  d4   f1
5  a5  b5  c5  d5   f2 

    a   b   c   d    f
2  a2  b2  c2  d2  NaN
3  a3  b3  c3  d3   f0
4  a4  b4  c4  d4   f1
5  a5  b5  c5  d5   f2 

     a    b    c    h
0   a0   b0   c0  NaN
1   a1   b1   c1   h0
2   a2   b2   c2   h1
3   a3   b3   c3   h2
4  NaN  NaN  NaN   h3


**시리즈들로 만든 리스트를 concat() 함수에 전달**하면 시리즈가 서로 연결된다. axis 매개변수를 1로 지정하면 데이터프레임, 0으로 지정하면 시리즈가 반환된다.

In [ ]:
result6 = pd.concat([sr1, sr3], axis = 1)
print(result6, '\n')

result7 = pd.concat([sr1, sr3], axis = 0)
print(result7)

    e   g
0  e0  g0
1  e1  g1
2  e2  g2
3  e3  g3 

0    e0
1    e1
2    e2
3    e3
0    g0
1    g1
2    g2
3    g3
dtype: object


<h3> 데이터프레임 병합 </h3>

판다스의 **merge() 함수**를 이용하면 어떤 기준에 의해 두 데이터프레임을 병합할 수 있다. 이때 기준이 되는 열이나 인덱스를 키(key)라고 부른다. 키가 되는 열이나 인덱스는 반드시 양쪽 데이터프레임에 존재해야 한다.

* **데이터프레임 병합: pandas.merge(df_left, df_right, how = 'inner', on = None)**
* how 매개변수에 가능한 옵션: 'inner', 'outer', 'left', 'right'

여러 종목의 주가 정보를 담고 있는 Excel 파일과 밸류에이션 정보를 정리한 Excel 파일을 불러오자.

In [ ]:
pd.set_option('display.max_columns', 10)                # 출력할 최대 열의 개수
pd.set_option('display.max_colwidth', 20)               # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True) # 유니코드 사용 너비 조정

# 주식 데이터를 가져온다.
df1 = pd.read_excel('stock price.xlsx', engine = 'openpyxl')
df2 = pd.read_excel('stock valuation.xlsx', engine = 'openpyxl')

print(df1)
print('\n')
print(df2)

       id    stock_name          value   price
0  128940      한미약품   59385.666667  421000
1  130960        CJ E&M   58540.666667   98900
2  138250    엔에스쇼핑   14558.666667   13200
3  139480        이마트  239230.833333  254500
4  142280  녹십자엠에스     468.833333   10200
5  145990        삼양사   82750.000000   82000
6  185750        종근당   40293.666667  100500
7  192400    쿠쿠홀딩스  179204.666667  177500
8  199800          툴젠   -2514.333333  115400
9  204210  모두투어리츠    3093.333333    3475


       id              name           eps     bps        per       pbr
0  130960            CJ E&M   6301.333333   54068  15.695091  1.829178
1  136480              하림    274.166667    3551  11.489362  0.887074
2  138040    메리츠금융지주   2122.333333   14894   6.313806  0.899691
3  139480            이마트  18268.166667  295780  13.931338  0.860437
4  145990            삼양사   5741.000000  108090  14.283226  0.758627
5  161390        한국타이어   5648.500000   51341   7.453306  0.820007
6  181710   NHN엔터테인먼트   2110.166667   784

두 데이터프레임을 병합해보자. how = 'inner', on = None 옵션이 기본값으로 적용된다. <br>
**how = 'inner' 옵션**은 기준 열의 데이터가 양쪽 데이터프레임에 공통으로 존재하는 (교집합일) 경우에만 추출한다는 의미이다. <br>
**on = None 옵션**은 두 데이터프레임에 공통으로 속하는 모든 열을 기준(키)으로 병합한다는 의미이다. <br>
위 데이터프레임들의 공통 열은 'id'하나 이고 'id'열에서 겹치는 데이터는 총 5개이다. 따라서 merge() 함수를 실행하면 총 5개의 행이 출력된다.

In [ ]:
# how = 'inner', on = None
merge_inner = pd.merge(df1, df2)
print(merge_inner)

       id    stock_name          value   price          name           eps  \
0  130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
1  139480        이마트  239230.833333  254500        이마트  18268.166667   
2  145990        삼양사   82750.000000   82000        삼양사   5741.000000   
3  185750        종근당   40293.666667  100500        종근당   3990.333333   
4  204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

      bps        per       pbr  
0   54068  15.695091  1.829178  
1  295780  13.931338  0.860437  
2  108090  14.283226  0.758627  
3   40684  25.185866  2.470259  
4    5335  40.802348  0.651359  


이번에는 how 매개변수를 'outer'로 지정해보자. 'outer'를 사용하면 기준 열의 데이터가 데이터프레임 중 한쪽에만 존재해도 포함한다. <br>
on 매개변수는 'id'열을 지정하자. 이는 두 데이터프레임의 공통 열 중에서 'id' 열을 키로 병합한다는 의미이다.

In [ ]:
# how = 'outer', on = 'id'
merge_outer = pd.merge(df1, df2, how = 'outer', on = 'id') 
print(merge_outer)

        id    stock_name          value     price              name  \
0   128940      한미약품   59385.666667  421000.0               NaN   
1   130960        CJ E&M   58540.666667   98900.0            CJ E&M   
2   138250    엔에스쇼핑   14558.666667   13200.0               NaN   
3   139480        이마트  239230.833333  254500.0            이마트   
4   142280  녹십자엠에스     468.833333   10200.0               NaN   
5   145990        삼양사   82750.000000   82000.0            삼양사   
6   185750        종근당   40293.666667  100500.0            종근당   
7   192400    쿠쿠홀딩스  179204.666667  177500.0               NaN   
8   199800          툴젠   -2514.333333  115400.0               NaN   
9   204210  모두투어리츠    3093.333333    3475.0      모두투어리츠   
10  136480           NaN            NaN       NaN              하림   
11  138040           NaN            NaN       NaN    메리츠금융지주   
12  161390           NaN            NaN       NaN        한국타이어   
13  181710           NaN            NaN       NaN   NHN엔터테인먼트   
14  207

how 매개변수를 **'left'**로 지정하면 왼쪽 데이터프레임의 키 열에 속하는 데이터 값을 기준으로 병합한다. 또한 **left_on, right_on 매개변수**를 사용해 좌우 데이터프레임에 각각 다르게 키를 지정할 수 있다.

In [ ]:
merge_left = pd.merge(df1, df2, how = 'left', left_on = 'stock_name', right_on = 'name')
print(merge_left)

     id_x    stock_name          value   price      id_y          name  \
0  128940      한미약품   59385.666667  421000       NaN           NaN   
1  130960        CJ E&M   58540.666667   98900  130960.0        CJ E&M   
2  138250    엔에스쇼핑   14558.666667   13200       NaN           NaN   
3  139480        이마트  239230.833333  254500  139480.0        이마트   
4  142280  녹십자엠에스     468.833333   10200       NaN           NaN   
5  145990        삼양사   82750.000000   82000  145990.0        삼양사   
6  185750        종근당   40293.666667  100500  185750.0        종근당   
7  192400    쿠쿠홀딩스  179204.666667  177500       NaN           NaN   
8  199800          툴젠   -2514.333333  115400       NaN           NaN   
9  204210  모두투어리츠    3093.333333    3475  204210.0  모두투어리츠   

            eps       bps        per       pbr  
0           NaN       NaN        NaN       NaN  
1   6301.333333   54068.0  15.695091  1.829178  
2           NaN       NaN        NaN       NaN  
3  18268.166667  295780.0  13.931338  0.8

how 매개변수를 **'right'**로 지정하면 오른쪽 데이터프레임의 키 열에 속하는 데이터 값을 기준으로 병합한다.

In [ ]:
merge_right = pd.merge(df1, df2, how = 'right', left_on = 'stock_name', right_on = 'name')
print(merge_right)

       id_x    stock_name          value     price    id_y              name  \
0  130960.0        CJ E&M   58540.666667   98900.0  130960            CJ E&M   
1       NaN           NaN            NaN       NaN  136480              하림   
2       NaN           NaN            NaN       NaN  138040    메리츠금융지주   
3  139480.0        이마트  239230.833333  254500.0  139480            이마트   
4  145990.0        삼양사   82750.000000   82000.0  145990            삼양사   
5       NaN           NaN            NaN       NaN  161390        한국타이어   
6       NaN           NaN            NaN       NaN  181710   NHN엔터테인먼트   
7  185750.0        종근당   40293.666667  100500.0  185750            종근당   
8  204210.0  모두투어리츠    3093.333333    3475.0  204210      모두투어리츠   
9       NaN           NaN            NaN       NaN  207940  삼성바이오로직스   

            eps     bps        per       pbr  
0   6301.333333   54068  15.695091  1.829178  
1    274.166667    3551  11.489362  0.887074  
2   2122.333333   14894   6.313806  

merge() 함수를 불린 인덱싱과 함께 사용하면 원하는 데이터를 추출할 수 있다.<br>
주가 50,000원 미만인 종목을 찾고 해당 종목의 밸류에이션 지표를 확인해보자. <br>

In [ ]:
price = df1[df1['price'] < 50000]
print(price.head())
print('\n')

value = pd.merge(price, df2)
print(value)
# 결과: 주가 50,000인 종목은 3개이고, 밸류에이션 데이터를 가진 회사는 1개이다.

       id    stock_name         value  price
2  138250    엔에스쇼핑  14558.666667  13200
4  142280  녹십자엠에스    468.833333  10200
9  204210  모두투어리츠   3093.333333   3475


       id    stock_name        value  price          name        eps   bps  \
0  204210  모두투어리츠  3093.333333   3475  모두투어리츠  85.166667  5335   

         per       pbr  
0  40.802348  0.651359  


<h3> 데이터프레임 결합 </h3>

판다스 **join() 메소드**는 merge() 함수를 기반으로 만들어져 기본 작동 방식이 비슷하다. 하지만 join() 메소드는 열이 아니라 행 인덱스를 기준으로 결합한다. 단, on 매개변수를 keys로 설정하면 열을 기준으로 결합할 수 있다.

* **행 인덱스 기준으로 결합: DataFrame1.join(DataFrame2, how = 'left')**

**index_col 옵션**을 사용해 두 데이터프레임의 행 인덱스를 'id'열로 설정하자. df1에 join() 메소드를 적용하면 왼쪽에 위치한 df1의 행 인덱스를 기준으로 결합하는 **how = 'left' 옵션**이 기본 적용된다.

In [ ]:
pd.set_option('display.max_columns', 10)                # 출력할 최대 열의 개수
pd.set_option('display.max_colwidth', 20)               # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True) # 유니코드 사용 너비 조정

# 주식 데이터를 가져온다.
df1 = pd.read_excel('stock price.xlsx', index_col = 'id', engine = 'openpyxl')
df2 = pd.read_excel('stock valuation.xlsx', index_col = 'id', engine = 'openpyxl')

df3 = df1.join(df2)
print(df3)

          stock_name          value   price          name           eps  \
id                                                                        
128940      한미약품   59385.666667  421000           NaN           NaN   
130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
138250    엔에스쇼핑   14558.666667   13200           NaN           NaN   
139480        이마트  239230.833333  254500        이마트  18268.166667   
142280  녹십자엠에스     468.833333   10200           NaN           NaN   
145990        삼양사   82750.000000   82000        삼양사   5741.000000   
185750        종근당   40293.666667  100500        종근당   3990.333333   
192400    쿠쿠홀딩스  179204.666667  177500           NaN           NaN   
199800          툴젠   -2514.333333  115400           NaN           NaN   
204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

             bps        per       pbr  
id                                     
128940       NaN        NaN       NaN  
130960   54068.0  15.695091  1.

how 매개변수를 'inner'로 설정하면 두 데이터프레임의 공통 행 인덱스를 기준으로 추출한다.

In [ ]:
df4 = df1.join(df2, how = 'inner')
print(df4)

          stock_name          value   price          name           eps  \
id                                                                        
130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
139480        이마트  239230.833333  254500        이마트  18268.166667   
145990        삼양사   82750.000000   82000        삼양사   5741.000000   
185750        종근당   40293.666667  100500        종근당   3990.333333   
204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

           bps        per       pbr  
id                                   
130960   54068  15.695091  1.829178  
139480  295780  13.931338  0.860437  
145990  108090  14.283226  0.758627  
185750   40684  25.185866  2.470259  
204210    5335  40.802348  0.651359  
